In [49]:
# importing necessary libraries
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelBinarizer

#TabNet librariees
from catboost import CatBoostClassifier

In [56]:
base_path = 'homeworks/data/project/gesture_phase/'
df = pd.read_csv(base_path + 'gesture_phase.csv')
X = df.values[:, :-1]
y_target = df.values[:, -1]
# Initialize the LabelEncoder
le = LabelEncoder()

# Fit and transform the data
y = le.fit_transform(y_target)

# binarizer to matix NxK
#lb = LabelBinarizer()
#y= lb.fit_transform(y)
#if y.shape[1] < 5:
#    y= np.hstack((y, np.zeros((y.shape[0], 5 - y.shape[1]))))

In [57]:

# Load Dataset

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Split into train and validation sets

# Paso 2: Dividir el 80% de entrenamiento + validación en 70% entrenamiento y 10% validación
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, random_state=42)



In [58]:
clf = CatBoostClassifier(
    iterations=500, 
    learning_rate=0.1, 
    random_seed=42,
    loss_function = "MultiClass",
    #eval_metric="HammingLoss",
    #class_names=["0", "1", "2", "3", "4"]
)


clf.fit(X_train, y_train, 
        #cat_features=cat_features, 
        eval_set=(X_val, y_val), 
        #verbose=True,
        plot = True
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.5399104	test: 1.5457171	best: 1.5457171 (0)	total: 21.2ms	remaining: 10.6s
1:	learn: 1.4888274	test: 1.5015973	best: 1.5015973 (1)	total: 35.2ms	remaining: 8.77s
2:	learn: 1.4465808	test: 1.4619213	best: 1.4619213 (2)	total: 48.8ms	remaining: 8.08s
3:	learn: 1.4110490	test: 1.4294973	best: 1.4294973 (3)	total: 62.1ms	remaining: 7.7s
4:	learn: 1.3794567	test: 1.4029287	best: 1.4029287 (4)	total: 82.4ms	remaining: 8.16s
5:	learn: 1.3543310	test: 1.3800321	best: 1.3800321 (5)	total: 94.8ms	remaining: 7.8s
6:	learn: 1.3332965	test: 1.3617593	best: 1.3617593 (6)	total: 107ms	remaining: 7.55s
7:	learn: 1.3136135	test: 1.3464891	best: 1.3464891 (7)	total: 118ms	remaining: 7.29s
8:	learn: 1.2953873	test: 1.3322196	best: 1.3322196 (8)	total: 129ms	remaining: 7.05s
9:	learn: 1.2784925	test: 1.3186594	best: 1.3186594 (9)	total: 143ms	remaining: 7.02s
10:	learn: 1.2658124	test: 1.3100974	best: 1.3100974 (10)	total: 156ms	remaining: 6.95s
11:	learn: 1.2534265	test: 1.3010383	best: 1.301

In [59]:
#print(clf.predict_proba(X_test))
#print(clf.predict(X_test))
print(clf.tree_count_)

498


In [60]:
from catboost.utils import eval_metric
test_predict = clf.predict(X_test)
accuracy = eval_metric(y_test, test_predict, 'Accuracy')[0]
print(f'Accuracy: {accuracy}')

accuracy_per_class = eval_metric(y_test, test_predict, 'Accuracy:type=PerClass')
for cls, value in zip(clf.classes_, accuracy_per_class):
    print(f'Accuracy for class {cls}: {value}')

Accuracy: 0.8526582278481013
Accuracy for class 0: 0.8526582278481013


In [64]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
# printing metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.6582278481012658
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.84      0.76       551
           1       0.65      0.40      0.50       197
           2       0.63      0.55      0.59       418
           3       0.52      0.41      0.46       215
           4       0.67      0.74      0.71       594

    accuracy                           0.66      1975
   macro avg       0.63      0.59      0.60      1975
weighted avg       0.65      0.66      0.65      1975



In [47]:
hamming = eval_metric(y_test, test_predict, 'HammingLoss')[0]
print(f'HammingLoss: {hamming:.4f}')
mean_accuracy_per_class = sum(accuracy_per_class) / len(accuracy_per_class)
print(f'MeanAccuracyPerClass: {mean_accuracy_per_class:.4f}')
print(f'HammingLoss + MeanAccuracyPerClass = {hamming + mean_accuracy_per_class}')

HammingLoss: 0.1473
MeanAccuracyPerClass: 0.8527
HammingLoss + MeanAccuracyPerClass = 1.0


In [48]:
print(clf.classes_)
for metric in ('Precision', 'Recall', 'F1'):
    print(metric)
    values = eval_metric(y_test, test_predict, metric)
    for cls, value in zip(clf.classes_, values):
        print(f'class={cls}: {value:.4f}')
    print()

['0' '1' '2' '3' '4']
Precision
class=0: 0.9341

Recall
class=0: 0.8560

F1
class=0: 0.8934



MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

KeyboardInterrupt: 